In [33]:
import requests
import pandas as pd
import numpy as np

BASE_API_URL = "https://historical-forecast-api.open-meteo.com/v1/forecast?"

# paramters
TORONTO_COORDINATES = "latitude=43.7064&longitude=-79.3986&"
DATE_RANGE_1 = "start_date=2024-01-01&end_date=2024-12-31&"
DATE_RANGE_2 = "start_date=2023-01-01&end_date=2023-12-31&"

COLUMNS = "hourly=temperature_2m,dew_point_2m,precipitation_probability,precipitation,weather_code,wind_speed_120m,wind_direction_80m,visibility,relative_humidity_2m"

API_URL_1 = BASE_API_URL + TORONTO_COORDINATES + TORONTO_COORDINATES + DATE_RANGE_1 + COLUMNS
API_URL_2 = BASE_API_URL + TORONTO_COORDINATES + TORONTO_COORDINATES + DATE_RANGE_2 + COLUMNS


In [34]:
data_1 = requests.get(API_URL_1)
data_2 = requests.get(API_URL_2)

In [35]:
weather_df_1 = pd.DataFrame(data_1.json()[0]["hourly"])
weather_df_2 = pd.DataFrame(data_2.json()[0]["hourly"])
weather_df = pd.concat([weather_df_1, weather_df_2])

/tmp/ipykernel_55176/846414910.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_df = pd.concat([weather_df_1, weather_df_2])


In [36]:
# datetime formatting 
weather_df["time"] = pd.to_datetime(weather_df["time"])
weather_df["Month"] = weather_df["time"].dt.month
weather_df["DayOfWeek"] = weather_df["time"].dt.dayofweek
weather_df["day"] = weather_df["time"].dt.day
weather_df["hour"] = weather_df["time"].dt.hour

# so that january and december are not seen as far apart
weather_df['month_cos'] = np.cos(2 * np.pi * weather_df['Month'] / 12)

weather_df['day_cos'] = np.cos(2 * np.pi * weather_df['DayOfWeek'] / 7)


weather_df

,time,temperature_2m,dew_point_2m,precipitation_probability,precipitation,weather_code,wind_speed_120m,wind_direction_80m,visibility,relative_humidity_2m,Month,DayOfWeek,day,hour,month_cos,day_cos
0,2024-01-01 00:00:00,-0.6,-1.0,NaN,0.2,73,12.8,50,12200.0,97,1,0,1,0,0.866025,1.00000
1,2024-01-01 01:00:00,-0.5,-1.4,NaN,0.2,73,14.6,52,13000.0,94,1,0,1,1,0.866025,1.00000
2,2024-01-01 02:00:00,-0.7,-1.7,NaN,0.3,73,15.8,44,8000.0,93,1,0,1,2,0.866025,1.00000
3,2024-01-01 03:00:00,-1.0,-1.9,NaN,0.3,73,16.0,39,12500.0,94,1,0,1,3,0.866025,1.00000
4,2024-01-01 04:00:00,-1.4,-2.4,NaN,0.7,73,15.6,47,11100.0,93,1,0,1,4,0.866025,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00,0.6,-0.5,NaN,0.0,3,6.1,68,14600.0,92,12,6,31,19,1.000000,0.62349
8756,2023-12-31 20:00:00,0.3,-0.8,NaN,0.0,3,7.6,74,14200.0,92,12,6,31,20,1.000000,0.62349
8757,2023-12-31 21:00:00,0.1,-1.1,NaN,0.0,3,9.7,63,14100.0,92,12,6,31,21,1.000000,0.62349
8758,2023-12-31 22:00:00,0.0,-1.4,NaN,0.0,3,10.7,63,14800.0,90,12,6,31,22,1.000000,0.62349


In [37]:
weather_df.to_csv("api_weather_data_toronto.csv")